<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from backtesting.test import GOOG, SMA
from backtesting import Backtest, Strategy
from backtesting.lib import TrailingStrategy

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [11]:
class Simple(Strategy):
    n = 10
    def init(self):
        self.sma = self.I(SMA, self.data.Close, self.n)
    def next(self):
        pass
        
class SMATrailing(TrailingStrategy):
    n = 10
    def init(self):
        self.sma = self.I(SMA, self.data.Close, self.n)
        
bt = Backtest(GOOG, Simple,)
bt_trailing = Backtest(GOOG, SMATrailing)

# %debug bt_trailing.run()

In [20]:
from typing import Tuple, Union

from backtesting import Strategy, Backtest
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
from pandas import Series


class Simple(Strategy):
    n_fast = 3
    n_slow = 10
    tp_long = 1.05
    sl_long = 0.95
    tp_short = 0.95
    sl_short = 1.05

    def init(self):
        self.n_fast = self.I(SMA, self.data.Close, self.n_fast)
        self.n_slow = self.I(SMA, self.data.Close, self.n_slow)

    def next(self):

        if crossover(self.n_fast, self.n_slow):
            self.buy(sl=self._data.Close[-1] * self.sl_long, tp=self._data.Close[-1] * self.tp_long)

        if crossover(self.n_slow, self.n_fast):
            self.sell(tp=self._data.Close[-1] * self.tp_short, sl=self._data.Close[-1] * self.sl_short)


bt: Backtest = Backtest(GOOG, Simple, commission=.002, margin=.5)
s: Union[Series, Tuple[Series, Series], Tuple[Series, Series, dict]] = bt.optimize(
        n_fast=range(2, 5),
        n_slow=range(3, 6),
        tp_long=list(np.arange(1.01,2.0,.1)),
        sl_long=list(np.arange(.8,.95,0.05)),
        tp_short=list(np.arange(.8,.95,0.05)),
        sl_short=list(np.arange(1.01,2.0,.1)),
        constraint=lambda p: p.n_fast < p.n_slow
)

/home/sergey/anaconda3/lib/python3.7/site-packages/backtesting/backtesting.py:1469: UserWarning: Searching for best of 5400 configurations.
  output.append(res)


In [21]:
s

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     98.8827
Equity Final [$]                        95547
Equity Peak [$]                       97716.5
Return [%]                             855.47
Buy & Hold Return [%]                 703.458
Return (Ann.) [%]                     30.3159
Volatility (Ann.) [%]                 99.6464
Sharpe Ratio                         0.304235
Sortino Ratio                        0.650064
Calmar Ratio                         0.365991
Max. Drawdown [%]                    -82.8325
Avg. Drawdown [%]                    -9.27312
Max. Drawdown Duration      976 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                  210
Win Rate [%]                          76.1905
Best Trade [%]                        63.8926
Worst Trade [%]                      -24.7312
Avg. Trade [%]                    